##  最小化 $2x^2 + 2xy + y^2-10x-10y$ 
满足 $x^2 + y^2 ≤ 5$ 

$3x+y≤ 6$

In [1]:
import numpy as np
import ipyvolume.pylab as plt3d

In [2]:
f = lambda X: 2 * X[0]**2 + 2 * X[0] * X[1] + X[1]**2 - 10 * X[0] - 10 * X[1]
h1 = lambda X: -X[0]**2 - X[1]**2 + 5
h2 = lambda X: -3 * X[0] - X[1] + 6

In [3]:
xn = np.linspace(-5, 5, 50)
yn = np.linspace(-5, 5, 50)
xxn, yyn = np.meshgrid(xn, yn)
fn = f([xxn, yyn])
h1n = h1([xxn, yyn])
h2n = h2([xxn, yyn])

plt3d.figure(width=1000, height=600)
plt3d.plot_surface(xxn, fn, yyn, color='blue')
plt3d.plot_wireframe(xxn, h1n, yyn, color='yellow')
plt3d.plot_wireframe(xxn, h2n, yyn, color='green')
plt3d.ylabel("z")
plt3d.zlabel("y")
plt3d.show()

## KKT 条件

In [4]:
import sympy
sympy.init_printing()

In [5]:
x, y = sympy.symbols("x, y", real=True)
f_sym = 2 * x**2 + 2 * x * y + y**2 - 10 * x - 10 * y
h1_sym = -x**2 - y**2 + 5
h2_sym = -3 * x - y + 6

### 1. 平稳性

In [10]:
f_grad = [f_sym.diff(i) for i in (x, y)]
f_grad = sympy.Matrix(f_grad)

mu1, mu2 = sympy.symbols("mu1, mu2", real=True)
mu = sympy.Matrix([mu1, mu2])
H = sympy.Matrix([h1_sym, h2_sym])
H_grad = [H.diff(i) for i in (x, y)]
H_grad = sympy.Matrix(H_grad).reshape(2, 2)

stat_cond = f_grad - H_grad * mu
stat_cond

⎡2⋅μ₁⋅x + 3⋅μ₂ + 4⋅x + 2⋅y - 10⎤
⎢                              ⎥
⎣ 2⋅μ₁⋅y + μ₂ + 2⋅x + 2⋅y - 10 ⎦

### 2. 原始可行性

In [12]:
prim_h1 = sympy.simplify(h1_sym >= 0)
prim_h2 = sympy.simplify(h2_sym >= 0)
prim_cond = sympy.Matrix([prim_h1, prim_h2])
prim_cond

⎡ 2    2        ⎤
⎢x  + y  - 5 ≤ 0⎥
⎢               ⎥
⎣  3⋅x + y ≤ 6  ⎦

### 3. 互补松弛性

In [13]:
slack_cond = sympy.Matrix([mu1 * h1_sym, mu2 * h2_sym])
slack_cond

⎡   ⎛   2    2    ⎞⎤
⎢μ₁⋅⎝- x  - y  + 5⎠⎥
⎢                  ⎥
⎣μ₂⋅(-3⋅x - y + 6) ⎦

### 4. 对偶可行性

In [15]:
dual_mu = sympy.Matrix([mu1>=0, mu2>=0])
dual_mu

⎡μ₁ ≥ 0⎤
⎢      ⎥
⎣μ₂ ≥ 0⎦

### 组成最优性

In [16]:
optimal_cond = sympy.Matrix([stat_cond, slack_cond, prim_cond, dual_mu])
optimal_cond

⎡2⋅μ₁⋅x + 3⋅μ₂ + 4⋅x + 2⋅y - 10⎤
⎢                              ⎥
⎢ 2⋅μ₁⋅y + μ₂ + 2⋅x + 2⋅y - 10 ⎥
⎢                              ⎥
⎢         ⎛   2    2    ⎞      ⎥
⎢      μ₁⋅⎝- x  - y  + 5⎠      ⎥
⎢                              ⎥
⎢      μ₂⋅(-3⋅x - y + 6)       ⎥
⎢                              ⎥
⎢        2    2                ⎥
⎢       x  + y  - 5 ≤ 0        ⎥
⎢                              ⎥
⎢         3⋅x + y ≤ 6          ⎥
⎢                              ⎥
⎢            μ₁ ≥ 0            ⎥
⎢                              ⎥
⎣            μ₂ ≥ 0            ⎦

In [17]:
optimal_equ = sympy.Matrix([stat_cond, slack_cond])
optimal_equ

⎡2⋅μ₁⋅x + 3⋅μ₂ + 4⋅x + 2⋅y - 10⎤
⎢                              ⎥
⎢ 2⋅μ₁⋅y + μ₂ + 2⋅x + 2⋅y - 10 ⎥
⎢                              ⎥
⎢         ⎛   2    2    ⎞      ⎥
⎢      μ₁⋅⎝- x  - y  + 5⎠      ⎥
⎢                              ⎥
⎣      μ₂⋅(-3⋅x - y + 6)       ⎦

In [25]:
roots = sympy.solve(optimal_equ.subs(mu2, 0), (x, y, mu1), real=True)
roots_float = []
for root in roots:
    t = []
    for v in root:
        t.append(v.evalf())
    roots_float.append(t)
    
roots_float

In [32]:
# 获取实根
roots_real = roots_float.copy()
for i in range(len(roots_float)):
    if "I" in str(roots_float[i]):
        roots_real.remove(roots_float[i])
        
roots_real

### 检查

In [34]:
# 检查Hessian矩阵是否正定
f_hessian = sympy.Matrix([[f_sym.diff(x_, y_) for x_ in (x, y)] for y_ in (x, y)])
f_hessian.is_positive_definite

True

In [35]:
prim_h1.subs({x: real_roots[0][0], y: real_roots[0][1]})

False

In [36]:
dual_mu.subs({mu1: real_roots[2][2], mu2: 0})

⎡False⎤
⎢     ⎥
⎣True ⎦

In [55]:
final_root = real_roots[1]
f_min = f_sym.subs({x: 1, y: 2}).evalf()
print("最小值点: x={:.4f}, y={:.4f}, f_min={:.4f}".format(final_root[0], 
                                                      final_root[1],
                                                      f_min))

最小值点: x=1.0000, y=2.0000, f_min=-20.0000


In [40]:
final_root = np.array(final_root).astype("float")

plt3d.figure(width=1000, height=600)
plt3d.plot_surface(xxn, fn, yyn, color='blue')
plt3d.plot_wireframe(xxn, h1n, yyn, color='yellow')
plt3d.plot_wireframe(xxn, h2n, yyn, color='green')
plt3d.scatter(np.array([final_root[0]]), np.array([float(f_min)]), 
              np.array([final_root[1]]), marker="diamond", size=5, color="red")
plt3d.ylabel("z")
plt3d.zlabel("y")
plt3d.show()

## 使用`Scipy`

In [42]:
from scipy import optimize
from scipy.optimize import LinearConstraint, NonlinearConstraint

In [57]:
nonlinear_constraint  = NonlinearConstraint(h1, 0, np.inf)
linear_constraint  = LinearConstraint([[3, 0], [0, 1]], 0, 6)
results = optimize.minimize(f, [0.1, 0.1],
                           constraints=[nonlinear_constraint, linear_constraint])
results

     fun: -20.000000001362327
     jac: array([-2., -4.])
 message: 'Optimization terminated successfully.'
    nfev: 37
     nit: 9
    njev: 9
  status: 0
 success: True
       x: array([1.00000001, 2.        ])

In [58]:
results.x, results.fun

(array([1.00000001, 2.        ]), -20.000000001362327)